In [1]:
import os, sys
import pandas as pd

In [2]:
ROOT_PATH = os.getenv('ROOT_PATH')
sys.path.append(ROOT_PATH)

In [3]:
from src.data_ingestion.iex_data import get_processed_data
from src.data_ingestion.weather_data import get_processed_weather
from config.utils import shift_date, capping
from config.paths import data_path
from src.feature_engineering.build_features import *

In [4]:
try:
    dam = pd.read_pickle(os.path.join(data_path, 'processed', 'dam_data'))
    rtm = pd.read_pickle(os.path.join(data_path, 'processed', 'rtm_data'))
    weather = pd.read_pickle(os.path.join(data_path, 'processed', 'weather_data'))
    wind = pd.read_pickle(os.path.join(data_path, 'processed', 'wind_data'))
    hydro = pd.read_pickle(os.path.join(data_path, 'processed', 'hydro_data'))
    solar = pd.read_pickle(os.path.join(data_path, 'processed', 'solar_data'))

except:
    dam = get_processed_data('dam')
    rtm = get_processed_data('rtm')
    weather = get_processed_weather('weather')
    wind = get_processed_weather('wind')
    hydro = get_processed_weather('hydro')
    solar = get_processed_weather('solar')

In [5]:
rtm = shift_date(rtm, 1) 
weather = shift_date(weather, -1)
hydro = shift_date(hydro, -1) 
solar = shift_date(solar, -1) 
wind = shift_date(wind, -1) 

In [6]:
data = merge_dataframes([dam, rtm, weather, hydro, solar, wind])

In [7]:
data.reset_index(drop=True)

,datetime,mcp_dam,clearedvolume_dam,pb_dam,sb_dam,diff_sb_pb_dam,mcp_rtm,clearedvolume_rtm,pb_rtm,sb_rtm,...,ws_van,wd_van,ws_bra,wd_bra,ws_dha,wd_dha,ws_jai,wd_jai,ws_mup,wd_mup
0,2020-07-02 00:00:00,2649.42,7494.00,8723.7,9725.5,-1001.8,1749.01,658.90,676.5,2126.5,...,1.935416,297.991242,5.601816,149.999822,5.950755,272.493902,5.126044,61.340698,2.846220,143.262489
1,2020-07-02 00:15:00,2649.86,7318.20,8548.0,9814.5,-1266.5,999.98,642.50,646.6,2249.6,...,1.950825,296.550686,5.960809,199.867645,5.915761,276.229016,4.085061,70.199665,3.070810,193.916360
2,2020-07-02 00:30:00,2649.43,6996.50,8225.9,9965.5,-1739.6,2005.45,637.70,1150.1,2137.4,...,1.966233,295.110130,6.319803,249.735468,5.880766,279.964129,3.044078,79.058632,3.295399,244.570232
3,2020-07-02 00:45:00,2569.14,6608.80,7830.3,10146.0,-2315.7,2003.55,638.00,1108.1,2150.7,...,1.920152,296.873877,6.290173,249.609410,5.770141,279.938774,2.945603,81.294188,3.287538,244.542489
4,2020-07-02 01:00:00,2549.60,5989.80,7193.0,10214.8,-3021.8,2500.08,672.70,1291.7,1924.7,...,1.874071,298.637624,6.260544,249.483351,5.659516,279.913420,2.847128,83.529744,3.279678,244.514746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123717,2024-01-12 22:45:00,3224.73,5935.50,9424.0,15910.5,-6486.5,3470.36,2926.44,3865.4,9186.2,...,0.720000,317.000000,1.695000,262.250000,0.967500,97.500000,0.245000,334.750000,3.537500,20.500000
123718,2024-01-12 23:00:00,3160.38,6112.77,9693.6,16877.0,-7183.4,3470.77,2614.96,3936.6,9426.9,...,0.660000,321.000000,1.750000,263.500000,0.935000,180.000000,0.240000,326.500000,3.575000,20.000000
123719,2024-01-12 23:15:00,3142.08,6115.90,9539.9,17534.4,-7994.5,3391.40,2702.60,3919.7,9983.9,...,0.600000,325.000000,1.805000,264.750000,0.902500,262.500000,0.235000,318.250000,3.612500,19.500000
123720,2024-01-12 23:30:00,3130.16,5977.68,9350.9,17850.2,-8499.3,3200.10,2605.81,3943.9,10292.8,...,0.540000,329.000000,1.860000,266.000000,0.870000,345.000000,0.230000,310.000000,3.650000,19.000000


In [8]:
data = capping(data)

In [9]:
data_for_training = feature_engineering(data, weather, data_type = 'dam')

In [13]:
save_features(data_for_training, os.path.join(data_path, 'processed', 'training_data'))